In [133]:
import numpy as np
import pandas as pd
import scipy
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import re
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from IPython import get_ipython

### Основная функция создания всевозможных множеств

In [134]:
def get_powerset(some_list):
    """Returns all subsets of size 0 - len(some_list) for some_list"""
    if len(some_list) == 0:
        return [[]]

    subsets = []
    first_element = some_list[0]
    remaining_list = some_list[1:]
    # Strategy: get all the subsets of remaining_list. For each
    # of those subsets, a full subset list will contain both
    # the original subset as well as a version of the subset
    # that contains first_element
    for partial_subset in get_powerset(remaining_list):
        subsets.append(partial_subset)
        subsets.append(partial_subset[:] + [first_element])

    return subsets

Зададим для примера словарик такой

In [135]:
list = {'i' :[1,1,1,2,2,2,3,3], 'j' :[1,2,3,2,3,4,4,5]}
list = pd.DataFrame(list)
list

,i,j
0,1,1
1,1,2
2,1,3
3,2,2
4,2,3
5,2,4
6,3,4
7,3,5


In [136]:
def sortByLength(inputStr):
        return len(inputStr)
    
def sets(Frame):
    l = []
    for i in set(Frame[Frame.columns[0]]):
        l.append( set(Frame[Frame[Frame.columns[0]] == i][Frame.columns[1]]) )
    return l

Вот получили 3 множества

In [137]:
all_sets = sets(list)
all_sets

[{1, 2, 3}, {2, 3, 4}, {4, 5}]

### Функция пересечения множеств в каждой строчке всевозможных множеств

In [138]:
def set_of_row(subsets_of_all_sets, row):
    if( len(subsets_of_all_sets[row]) >= 2):
        a = subsets_of_all_sets[row][0].intersection(subsets_of_all_sets[row][1])
        for j in range(1,len(subsets_of_all_sets[row]) - 1):
            a = a.intersection(subsets_of_all_sets[row][j+1])
        return(a)
    if (len(subsets_of_all_sets[row]) == 1):
        return(subsets_of_all_sets[row][0])
    else:
         return(set())

- По началу думал сделать както через функцию, но потом понял как проще считать множества уникальные, то есть в которые входят только эти элементы и не входят ни в какие другие пересечения множеств той же длинны

In [139]:
'''
def func(all_sets, a):
    all_a = set()
    for i in range(len(all_sets)):
        if(a != all_sets[i]):
            all_a = all_a.union(all_sets[i])
        
    rez = a - all_a&a
        
    return(rez)
'''

'\ndef func(all_sets, a):\n    all_a = set()\n    for i in range(len(all_sets)):\n        if(a != all_sets[i]):\n            all_a = all_a.union(all_sets[i])\n        \n    rez = a - all_a&a\n        \n    return(rez)\n'

### Функция, которая создает массив множеств всевозможных подмножеств

In [140]:
def all_subsets(all_sets):
    l = []
    
    subsets_of_all_sets = get_powerset(all_sets)
    #subsets_of_all_sets.sort(key = sortByLength)
    
    for i in range(len(subsets_of_all_sets) ):
        l.append(set_of_row(subsets_of_all_sets, i))
    
    #for i in range(len(l)):
        #if l[i] in sets(list):
            #all_sets = sets(list)
            #l[i] = func(all_sets, l[i])
    all_sets = sets(list)
    return(l)

In [141]:
massiv = all_subsets(all_sets)
massiv

[set(), {1, 2, 3}, {2, 3, 4}, {2, 3}, {4, 5}, set(), {4}, set()]

## Зададим теперь словарик для этих пересечений, чтобы было удобно отображать информацию!

In [142]:
dict = ['A', 'B', 'C']
massiv_name = get_powerset(dict)
massiv_name = [str(i) for i in massiv_name]
massiv_name = [re.sub(r'\W', "", i) for i in massiv_name]
massiv_name[0] = 'Пустое множество'

In [143]:
massiv_name

['Пустое множество', 'A', 'B', 'BA', 'C', 'CA', 'CB', 'CBA']

In [144]:
def TABLE(massiv_name, massiv):
    table = pd.DataFrame([str(i) for i in massiv])
    table.index = massiv_name
    return(table.T)

### Протестируем на больших теперь множествах

In [145]:
all_sets = [{1,2,43,5}, {8,5,6,7}, {8,3222,0,-1,55}, {1}, {43,-1}]
dict = dict = ['A', 'B', 'C', 'D', 'E']

In [146]:
massiv = all_subsets(all_sets)
massiv_name = get_powerset(dict)
massiv_name = [str(i) for i in massiv_name]
massiv_name = [re.sub(r'\W', "", i) for i in massiv_name]
massiv_name[0] = 'Пустое множество'

In [147]:
TABLE1 = TABLE(massiv_name, massiv).drop(columns = ['Пустое множество'])
TABLE1

,A,B,BA,C,CA,CB,CBA,D,DA,DB,...,ECB,ECBA,ED,EDA,EDB,EDBA,EDC,EDCA,EDCB,EDCBA
0,"{1, 2, 43, 5}","{8, 5, 6, 7}",{5},"{0, 8, 3222, 55, -1}",set(),{8},set(),{1},{1},set(),...,set(),set(),set(),set(),set(),set(),set(),set(),set(),set()


In [148]:
TABLE =pd.DataFrame(index = np.arange(np.shape(TABLE1)[1]))
TABLE['&'] = TABLE1.T.index
TABLE['set'] = TABLE1.T[0].values
TABLE

,&,set
0,A,"{1, 2, 43, 5}"
1,B,"{8, 5, 6, 7}"
2,BA,{5}
3,C,"{0, 8, 3222, 55, -1}"
4,CA,set()
5,CB,{8}
6,CBA,set()
7,D,{1}
8,DA,{1}
9,DB,set()


### ПО сути у нас есть таблица, где есть теперь все произведения в порядке возрастания длинны
- Прикол в том, что $C^{k}_{N}$ - число вхождений подмножеств длины k в общее число множеств
 - А тогда уникальные элементы: 

           $set(a_1*...*a_n) = a1*...*a_n - \sum\limits(sets по всем длинам = n+1) a_i_1*....*a_i_*a_n+1 )$

In [149]:
TABLE = TABLE.iloc[TABLE.agg({"&":len}).sort_values('&').index]
TABLE.index = np.arange(len(TABLE))
TABLE.loc[TABLE['set'] == 'set()', 'set'] = '{}'
TABLE

,&,set
0,A,"{1, 2, 43, 5}"
1,D,{1}
2,C,"{0, 8, 3222, 55, -1}"
3,E,"{43, -1}"
4,B,"{8, 5, 6, 7}"
5,CA,{}
6,ED,{}
7,EC,{-1}
8,BA,{5}
9,EA,{43}


In [150]:
def f(n):
    if(n == 0):
        return 1
    else:
        return n*f(n-1)
def C(k,n):
    return( int(f(n) / f(k) / f(n-k)) )

In [151]:
mass_index_count = [C(i,len(all_sets)) for i in range(1,len(all_sets)+1)]

In [152]:
mass_index_star_end = np.zeros((len(all_sets), 2))
mass_index_star_end[0][1] = int(mass_index_count[0] - 1)
mass_index_star_end[0][0] = int(0)

for i in range(1,len(all_sets)-1):

    mass_index_star_end[i][1] = int(mass_index_star_end[i-1][1] + mass_index_count[i])
    mass_index_star_end[i][0] = int(mass_index_star_end[i-1][1]+1)

mass_index_star_end[len(all_sets)-1][1] = int(mass_index_star_end[i][1] + 1)
mass_index_star_end[len(all_sets)-1][0] = int(mass_index_star_end[i][1] + 1)

In [153]:
mass_index_star_end = mass_index_star_end.astype(int)
mass_index_star_end 

array([[ 0,  4],
       [ 5, 14],
       [15, 24],
       [25, 29],
       [30, 30]])

#### Проблема ДатаФрейма в том, что сеты которые я закинул туда перешли без ведома в string, пришлось их опять обратно переводить, а это оказалось не так просто как кажется, но я сделал кое как:
        a = re.sub(r',', "", TABLE['set'][i])
        a = re.sub(r'}', "", a)
        a = re.sub(r'{', "", a)
        a = [int(x) for x in a.split()]
        a = set(a)

In [154]:
def function_mult_sets_len(lenght, TABLE):
    '''Произведений длины N+1 будет mass_index_count[N]'''
    '''Индексы для произведений будут находиться между
    mass_index_star_end[N][0] и  mass_index_star_end[N][1]'''
    all = set()
    for i in range( mass_index_star_end[lenght][0], mass_index_star_end[lenght ][1] + 1):
        
        a = re.sub(r',', "", TABLE['set'][i])
        a = re.sub(r'}', "", a)
        a = re.sub(r'{', "", a)
        a = [int(x) for x in a.split()]
        a = set(a)
        
        all = all|a
    return all

# lenght > 1

def rezult_set_len(lenght, TABLE):
    '''Произведений длины N+1 будут вычтены из всех ячеек на размер меньшей длинны (N)'''
    """То есть мы изменил все сеты длины N"""
    for i in range(mass_index_star_end[lenght - 1][0], mass_index_star_end[lenght - 1][1] + 1):
        
        a = re.sub(r',', "", TABLE['set'][i])
        a = re.sub(r'}', "", a)
        a = re.sub(r'{', "", a)
        a = [int(x) for x in a.split()]
        a = set(a)
        
        TABLE['set'][i] =  a - function_mult_sets_len(lenght, TABLE)
    return TABLE

In [155]:
def FINAL(TABLE):
    for i in range(1, len(all_sets)):
        rezult_set_len(i, TABLE)
    return TABLE

In [156]:
FINAL(TABLE)

,&,set
0,A,{2}
1,D,{}
2,C,"{0, 3222, 55}"
3,E,{}
4,B,"{6, 7}"
5,CA,{}
6,ED,{}
7,EC,{-1}
8,BA,{5}
9,EA,{43}
